# Part 01: Import and export the street data
michael babb  
2024 11 24

In [ ]:
# standard
import os

In [ ]:
# external
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
# custom
import run_constants as rc
from utils import create_city_sector, write_gdf, create_name

# load the street network shapefile as a gpd

In [ ]:
# setup the output paths
if not os.path.exists(rc.OUTPUT_FILE_PATH):
    os.makedirs(rc.OUTPUT_FILE_PATH)

In [ ]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S01_SND_IN_FILE_NAME)

In [ ]:
gdf = gpd.read_file(filename = fpn)

In [ ]:
gdf.shape

# lower case and rename columns

In [ ]:
gdf.columns = [cn.lower() for cn in gdf.columns]

In [ ]:
gdf.columns

# change datatypes

In [ ]:
for cn in gdf.columns:
    if gdf[cn].dtype == np.int32:
        gdf[cn] = gdf[cn].astype(np.int64)
    

# indicate the sector of the city the street is in

In [ ]:
# combine the ord_pre_dir and the ord_suf_dir to get the sector of the city.

In [ ]:
col_names = ['ord_pre_dir', 'ord_suf_dir']

In [ ]:
for cn in col_names:
    gdf[cn] = gdf[cn].fillna('')

In [ ]:
gdf['city_sector'] = gdf.apply(func = create_city_sector, axis = 1)

In [ ]:
gdf['city_sector'].unique()

## note: I used qGIS to manually recode the following streets:
MAGNOLIA BR WB: W  
WEST SEATTLE BR WB: mix of SW and S  
WEST SEATTLE BR EB: mix of SW and S  
1ST AV S BR SB: mix of SW and S  
1ST AV S BR NB: mix of SW and S  
(it's a lot easier to do this in qGIS)

In [ ]:
check = gdf.loc[gdf['city_sector'].str[-1] == 'B', 'ord_stname_concat'].unique().tolist()

In [ ]:
check

In [ ]:
# let's create a unique name field

In [ ]:
gdf['ord_stname_unique'] = gdf.apply(func = create_name, axis = 1)

In [ ]:
gdf['ord_stname_concat'].unique().shape

In [ ]:
gdf['ord_stname_unique'].unique().shape

# write the geodataframe to disk as a gpkg

In [ ]:
# reproject to wgs 84
gdf = gdf.to_crs(crs = 'epsg:4326')

In [ ]:
write_gdf(gdf = gdf, output_file_path=rc.OUTPUT_FILE_PATH,
           output_file_name=rc.S01_SND_OUT_FILE_NAME)

# export the blank street types to an excel file for manual inspection

In [ ]:
gdf['ord_street_type'].unique()

In [ ]:
blank_street_type_df = gdf.loc[gdf['ord_street_type'].isna(), 'ord_stname_concat'].drop_duplicates().to_frame()

In [ ]:
blank_street_type_df.shape

In [ ]:
ofpn = os.path.join(rc.ANALYSIS_OUTPUT_FILE_PATH, rc.S01_BST_OUT_FILE_NAME)

In [ ]:
blank_street_type_df.to_excel(excel_writer=ofpn, index = False)